<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/DEEPCTR02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



深度兴趣网络（Deep Interest Network, DIN）是阿里巴巴在2018年提出的一种用于点击率预测（CTR）的深度学习模型。其核心思想可以用一个生活化的比喻来理解：想象你是一位网购爱好者，平台需要根据你过去浏览的商品（比如运动鞋、咖啡机、小说）来猜测你现在可能对哪个广告感兴趣。传统方法就像把所有历史行为“一视同仁”地分析，但DIN却能像人类一样，**动态关注与当前广告最相关的行为**。例如，当你看到一款新跑鞋广告时，DIN会重点参考你过去浏览运动鞋的记录，而忽略咖啡机和小说这类无关行为。

### 一、DIN解决了什么问题？
点击率预测（CTR）是推荐系统和广告投放的核心任务，目标是预测用户点击某个内容（如广告、商品）的概率。传统模型（如逻辑回归、因子分解机）存在两大局限：
1. **兴趣建模僵化**：用户的历史行为被压缩成固定长度的向量，无法灵活表达多样的兴趣。例如，用户可能同时喜欢“运动”和“文学”，但传统模型难以区分这两种兴趣在不同场景下的权重。
2. **噪声干扰**：用户行为中混杂大量无关历史（例如误点或随意浏览），传统模型无法有效过滤这些噪声。

### 二、DIN的核心创新：像人一样“动态关注”
DIN通过以下三个关键设计解决上述问题：
1. **注意力机制（Attention）**  
   这是DIN的灵魂。模型会为每个用户行为计算一个“相关性权重”：与当前广告越相关的行为，权重越高。例如，用户的历史行为包括“运动鞋、咖啡机、小说”，当预测跑鞋广告的点击率时，模型会给“运动鞋”行为赋予高权重，而“咖啡机”和“小说”的权重则很低。这就像人类看到广告时，只会回想相关的购买经历。

2. **自适应激活函数（Dice）**  
   传统激活函数（如ReLU）的阈值是固定的，但DIN的Dice函数能根据数据分布动态调整阈值。例如，当用户行为数据差异较大时，Dice会自动适应不同场景，提升模型的灵活性。

3. **高效正则化（Mini-batch Aware Regularization）**  
   面对海量数据，传统正则化方法计算成本极高。DIN只对当前训练批次（mini-batch）中出现过的特征进行正则化，既防止过拟合，又大幅减少计算量。

### 三、DIN的实际效果如何？
实验表明，DIN在多个场景下显著优于传统模型：
- **离线测试**：在亚马逊和MovieLens数据集上，DIN的AUC（衡量预测准确性的指标）比传统模型（如Wide&Deep、PNN）提升约1.89%。
- **线上应用**：在阿里巴巴广告系统中，DIN使点击率（CTR）提升10%，广告收入增长3.8%。这意味着每展示100次广告，DIN能多带来1次点击，这在亿级流量场景下效益巨大。

### 四、DIN的启示：从“静态画像”到“动态兴趣”
DIN的成功揭示了推荐系统的未来方向：**用户的兴趣是多样且动态变化的**。与其用固定标签定义用户（如“运动爱好者”），不如根据具体场景实时捕捉兴趣焦点。这种思路也被后续模型（如DIEN）进一步扩展，加入了兴趣演化的时序建模。

总结来说，DIN的核心思想是 **“动态相关性”** ——让模型像人一样，在不同场景下灵活关注最相关的历史行为，从而更精准地预测用户的点击意愿。这一创新不仅提升了技术指标，也推动了推荐系统从“粗放推荐”向“智能理解”的跨越。

In [ ]:
import torch

# 生成一个包含 10 个随机整数的一维张量，范围是 [0, 10)
random_tensor = torch.randint(0, 10, (10,))
print(random_tensor)

# 生成一个 3x2 的二维张量，范围是 [1, 5)
random_tensor_2d = torch.randint(1, 5, (3, 2))
print(random_tensor_2d)

tensor([0, 9, 8, 6, 5, 7, 1, 7, 8, 8])
tensor([[3, 4],
        [4, 3],
        [2, 1]])


**unsqueeze(1) 的作用？**

unsqueeze(1) 是 PyTorch 张量的一个方法，用于在指定维度上增加一个维度。1 表示在第二个维度（索引为 1）上增加维度。

例如，假设有一个形状为 (64,) 的张量 user_ages，表示 64 个用户的年龄。执行 user_ages.unsqueeze(1) 后，张量的形状会变成 (64, 1)。相当于把原来的一维张量变成了一个二维张量，其中每个元素都被放在一个单独的行中。

**为什么需要增加一个维度？**

在深度学习模型中，输入数据的维度通常需要满足特定的要求。例如，在 DIN 模型中，用户的年龄和商品价格都是数值型特征，通常会使用一个线性层来处理这些特征。线性层的输入需要是一个二维张量，其中第一维表示样本数量，第二维表示特征维度。

因此，如果用户的年龄或商品价格只有一维，就需要使用 unsqueeze(1) 在第二个维度上增加一个维度，使其变成一个二维张量，以便与线性层的输入维度匹配。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义模型的超参数
embedding_dim = 32  # Embedding 向量的维度，每个用户/商品/类别都会被映射到一个 32 维的向量
hidden_units = [64, 32]  # 全连接层隐藏单元数量，表示网络中两个隐藏层的节点数
num_users = 1000  # 用户总数
num_items = 2000  # 商品总数
num_categories = 10  # 商品类别总数


# 定义一个函数来生成模拟数据
def generate_sample_data(batch_size=64):
    # 生成用户 ID，范围在 [0, num_users) 之间，形状为 (batch_size,)
    user_ids = torch.randint(0, num_users, (batch_size,))
    # 生成用户性别，0 或 1，形状为 (batch_size,)
    user_genders = torch.randint(0, 2, (batch_size,))
    # 生成用户年龄，范围在 [18, 60) 之间，形状为 (batch_size,)，并转换为浮点数
    user_ages = torch.randint(18, 60, (batch_size,)).float()

    # 生成商品 ID，范围在 [0, num_items) 之间，形状为 (batch_size,)
    item_ids = torch.randint(0, num_items, (batch_size,))
    # 生成商品类别，范围在 [0, num_categories) 之间，形状为 (batch_size,)
    item_categories = torch.randint(0, num_categories, (batch_size,))
    # 生成商品价格，范围在 [0, 1000) 之间，形状为 (batch_size,)
    item_prices = torch.rand(batch_size) * 1000

    # 生成用户的历史行为序列，每个用户随机浏览 1-5 个商品
    history_item_ids_list = []
    for _ in range(batch_size):
        # 随机生成序列长度，范围在 [1, 6) 之间
        seq_len = torch.randint(1, 6, (1,)).item()
        # 生成历史行为序列，范围在 [0, num_items) 之间，形状为 (seq_len,)
        history_seq = torch.randint(0, num_items, (seq_len,))
        # 将生成的序列添加到列表中
        history_item_ids_list.append(history_seq)

    # 将历史行为序列填充到相同长度 (最长序列长度)，并转换为张量
    # 获取最长序列长度
    max_len = max([seq.size(0) for seq in history_item_ids_list])
    # 创建一个全零张量，形状为 (batch_size, max_len)，用于存储填充后的序列
    padded_history_item_ids = torch.zeros((batch_size, max_len), dtype=torch.long)
    # 遍历每个用户的历史行为序列
    for i, seq in enumerate(history_item_ids_list):
        # 将序列填充到 padded_history_item_ids 中
        padded_history_item_ids[i, :seq.size(0)] = seq

    # 生成目标商品 ID，范围在 [0, num_items) 之间，形状为 (batch_size,)
    target_item_ids = torch.randint(0, num_items, (batch_size,))
    # 生成点击标签，0 或 1，形状为 (batch_size,)，并转换为浮点数
    labels = torch.randint(0, 2, (batch_size,)).float()

    # 将所有数据打包成一个字典返回
    return {
        'user_id': user_ids,
        'user_gender': user_genders,
        'user_age': user_ages.unsqueeze(1),  # 增加一个维度，以便与模型输入匹配
        'item_id': item_ids,
        'item_category': item_categories,
        'item_price': item_prices.unsqueeze(1),  # 增加一个维度，以便与模型输入匹配
        'history_item_ids': padded_history_item_ids,
        'target_item_id': target_item_ids,
        'label': labels
    }

# 调用 generate_sample_data 函数生成一个 batch 的数据
sample_data = generate_sample_data()
# 打印示例数据的键值，以便查看数据结构
print("示例数据：", sample_data.keys())
# 打印历史行为序列的形状，以便查看数据维度
print("历史行为序列形状:", sample_data['history_item_ids'].shape)

示例数据： dict_keys(['user_id', 'user_gender', 'user_age', 'item_id', 'item_category', 'item_price', 'history_item_ids', 'target_item_id', 'label'])
历史行为序列形状: torch.Size([64, 5])


**nn.Embedding 的工作原理**

nn.Embedding 层的作用是将离散的特征（例如商品 ID）映射成连续的向量表示（embedding 向量）。它内部维护一个查找表，表中的每一行代表一个特征的 embedding 向量。

当我们输入一个特征 ID 时，nn.Embedding 层会根据这个 ID 在查找表中找到对应的 embedding 向量并返回。

**处理不同维度输入的关键**

nn.Embedding 层可以处理任意形状的整数张量作为输入。它会将输入张量中的每个元素都视为一个特征 ID，并根据 ID 查找对应的 embedding 向量。

处理 item_id: item_id 的形状是 (batch_size,)，它包含了每个样本的商品 ID。nn.Embedding 层会将 item_id 中的每个 ID 都映射成一个 embedding 向量，最终返回一个形状为 (batch_size, embedding_dim) 的张量。

处理 history_item_ids: history_item_ids 的形状是 (batch_size, seq_len)，它包含了每个样本的历史行为序列，序列中的每个元素都是一个商品 ID。nn.Embedding 层会将 history_item_ids 中的每个 ID 都映射成一个 embedding 向量，最终返回一个形状为 (batch_size, seq_len, embedding_dim) 的张量。

**总结**

nn.Embedding 层能够处理不同维度的输入，因为它会将输入张量中的每个元素都视为一个特征 ID，并根据 ID 查找对应的 embedding 向量。

因此，即使 item_id 和 history_item_ids 的维度不同，item_embedding 函数仍然可以处理它们，并返回相应形状的 embedding 向量张量。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DIN(nn.Module):
    def __init__(self, num_users, num_items, num_categories, embedding_dim, hidden_units):
        super(DIN, self).__init__()

        # 创建用户嵌入层，将用户ID映射到embedding_dim维度的向量空间
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        # 创建商品嵌入层，将商品ID映射到embedding_dim维度的向量空间
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        # 创建类别嵌入层，将类别ID映射到embedding_dim维度的向量空间
        self.category_embedding = nn.Embedding(num_categories, embedding_dim)

        # 创建数值型特征的线性层，将用户年龄和商品价格映射到embedding_dim维度的向量空间
        self.numerical_fc = nn.Linear(2, embedding_dim)

        # 创建注意力池化层，用于根据目标商品对历史行为进行加权
        self.attention_pooling = AttentionPoolingLayer(embedding_dim)

        # 创建全连接层，用于最终的点击率预测
        fc_layers = []
        # 初始化全连接层的输入维度，包括用户、商品、类别、数值型特征和注意力输出的embedding维度之和
        input_dim = embedding_dim * 4 + embedding_dim
        # 逐层构建全连接层，并使用ReLU作为激活函数
        for units in hidden_units:
            fc_layers.append(nn.Linear(input_dim, units))
            fc_layers.append(nn.ReLU())
            input_dim = units
        # 添加最后一层全连接层，输出维度为1，表示点击率预测值
        fc_layers.append(nn.Linear(input_dim, 1))
        # 将所有全连接层组合成一个序列
        self.fc = nn.Sequential(*fc_layers)


    def forward(self, user_id, user_gender, user_age, item_id, item_category, item_price, history_item_ids, target_item_id):
        """
        DIN模型的前向传播函数

        输入:
            user_id: 用户ID, 形状: (batch_size,)
            user_gender: 用户性别, 形状: (batch_size,)
            user_age: 用户年龄, 形状: (batch_size, 1)
            item_id: 商品ID, 形状: (batch_size,)
            item_category: 商品类别, 形状: (batch_size,)
            item_price: 商品价格, 形状: (batch_size, 1)
            history_item_ids: 用户历史行为序列, 形状: (batch_size, seq_len)
            target_item_id: 目标商品ID, 形状: (batch_size,)

        输出:
            output: 点击率预测值, 形状: (batch_size,)
        """
        # 1. 获取各个特征的嵌入向量
        # 获取用户嵌入向量
        user_embed = self.user_embedding(user_id)
        # 获取商品嵌入向量
        item_embed = self.item_embedding(item_id)
        # 获取类别嵌入向量
        category_embed = self.category_embedding(item_category)
        # 获取目标商品嵌入向量
        target_item_embed = self.item_embedding(target_item_id)
        # 获取历史行为序列嵌入向量
        history_item_embed = self.item_embedding(history_item_ids)


        # 2. 处理数值型特征
        # 将用户年龄和商品价格拼接成一个张量
        numerical_features = torch.cat([user_age, item_price], dim=-1)
        # 使用线性层和ReLU激活函数将数值型特征映射到embedding_dim维度的向量空间
        numerical_embed = F.relu(self.numerical_fc(numerical_features))

        # 3. 使用注意力池化层对历史行为进行加权
        # 使用注意力机制，根据目标商品，对用户的历史行为序列进行加权聚合，得到用户的兴趣表示
        attention_output = self.attention_pooling(queries=target_item_embed, keys=history_item_embed)

        # 4. 将所有特征拼接在一起
        # 将用户、商品、类别、数值型特征和注意力输出的embedding向量拼接在一起
        concat_features = torch.cat([user_embed, item_embed, numerical_embed, attention_output, target_item_embed], dim=-1)

        # 5. 通过全连接层进行预测
        # 将拼接后的特征向量输入到全连接层，得到预测结果
        output = self.fc(concat_features)
        # 使用 sigmoid 激活函数将输出转换为点击率 (0-1)
        output = torch.sigmoid(output)

        # 返回预测结果，并移除维度为 1 的维度
        return output.squeeze(1)


# 注意力池化层的实现（！存在问题！）
class AttentionPoolingLayer(nn.Module):
    def __init__(self, embedding_dim):
        super(AttentionPoolingLayer, self).__init__()
        # 创建一个线性层，将 embedding 向量映射到注意力权重
        self.attention_fc = nn.Linear(embedding_dim, 1)

    def forward(self, queries, keys):
        """
        前向传播函数，计算注意力权重并进行加权池化
        :param queries: 目标商品的嵌入向量 (batch_size, embedding_dim)
        :param keys: 历史行为序列的嵌入向量 (batch_size, seq_len, embedding_dim)
        :return: 注意力池化后的用户兴趣表示 (batch_size, embedding_dim)
        """

        # 1. 计算注意力分数
        # 扩展目标商品嵌入向量的维度，以便与历史行为序列嵌入向量进行元素乘法
        queries = queries.unsqueeze(1)  # (batch_size, 1, embedding_dim)
        # 计算目标商品与每个历史行为商品的相似度，作为注意力分数
        attention_scores = torch.sum(queries * keys, dim=-1)  # (batch_size, seq_len)

        # 2. 使用全连接层进一步学习注意力分数
        # 将历史行为序列嵌入向量进行reshape，然后通过线性层进行变换，得到新的注意力分数
        attention_scores = self.attention_fc(keys.view(-1, keys.size(-1))).view(keys.size(0), keys.size(1))  # (batch_size, seq_len)

        # 3. 使用 Softmax 归一化注意力分数，得到注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1)  # (batch_size, seq_len)

        # 4. 加权求和得到注意力池化后的结果
        # 扩展注意力权重的维度，与历史行为序列嵌入向量相乘，然后在序列维度上求和
        attention_output = torch.sum(attention_weights.unsqueeze(-1) * keys, dim=1)  # (batch_size, embedding_dim)

        return attention_output

# 初始化 DIN 模型
model = DIN(num_users, num_items, num_categories, embedding_dim, hidden_units)
print(model) # 打印模型结构

DIN(
  (user_embedding): Embedding(1000, 32)
  (item_embedding): Embedding(2000, 32)
  (category_embedding): Embedding(10, 32)
  (numerical_fc): Linear(in_features=2, out_features=32, bias=True)
  (attention_pooling): AttentionPoolingLayer(
    (attention_fc): Linear(in_features=32, out_features=1, bias=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=160, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


**queries * keys**: 这是两个张量的元素乘法操作。

queries 代表目标商品的 embedding 向量，形状为 (batch_size, 1, embedding_dim)。
keys 代表用户历史行为序列中每个商品的 embedding 向量，形状为 (batch_size, seq_len, embedding_dim)。

元素乘法操作会将 queries 和 keys 中对应位置的元素相乘，得到一个新的张量，形状为 (batch_size, seq_len, embedding_dim)。

**torch.sum(...)**: 这是对张量进行求和的操作。

dim=-1 指定了求和的维度，-1 表示最后一个维度，也就是 embedding_dim 这个维度。
因此，torch.sum(queries * keys, dim=-1) 会将 queries * keys 这个张量在 embedding_dim 维度上进行求和，得到一个新的张量，形状为 (batch_size, seq_len)。


**整体含义**:

这行代码的整体含义是计算目标商品与每个历史行为商品之间的相似度，作为注意力分数。

首先，通过元素乘法 (queries * keys) 计算目标商品 embedding 与每个历史行为商品 embedding 的对应元素乘积。

然后，通过在 embedding_dim 维度上求和 (torch.sum(...)) 得到一个标量值，这个标量值代表了目标商品与某个历史行为商品之间的相似度。

最终得到的张量 attention_scores 形状为 (batch_size, seq_len)，其中每个元素都代表目标商品与对应历史行为商品的相似度（注意力分数）。

**直观理解**：

可以将 queries 看作是目标商品的“查询向量”，将 keys 看作是历史行为商品的“键向量”。torch.sum(queries * keys, dim=-1) 的作用就是计算“查询向量”与每个“键向量”之间的相似度，相似度越高，注意力分数就越高，说明目标商品与该历史行为商品越相关。

In [ ]:
# 使用示例数据进行一次前向预测
model.eval() # 设置模型为评估模式 (不进行梯度计算)
with torch.no_grad(): # 上下文管理器，禁止梯度计算
    input_data = {k: v for k, v in sample_data.items() if k != 'label'}
    predictions = model(**input_data) # 将 input_data 字典作为参数传入模型
    print("预测结果 (前 10 个样本):", predictions[:10])
    print("预测结果形状:", predictions.shape)

预测结果 (前 10 个样本): tensor([1.6251e-01, 1.7627e-04, 2.3034e-01, 4.2826e-01, 5.1056e-04, 1.3396e-01,
        4.6400e-03, 2.7882e-03, 3.0229e-02, 1.2459e-01])
预测结果形状: torch.Size([64])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import roc_auc_score

# 定义模型的超参数
embedding_dim = 32   # Embedding 向量的维度
hidden_units = [64, 32]   # 全连接层隐藏单元数量
num_users = 1000   # 用户总数
num_items = 2000   # 商品总数
num_categories = 50   # 商品类别总数
num_genders = 2 # 用户性别数量

# 定义一个函数来生成模拟数据
def generate_sample_data(batch_size=64):
    # 生成用户 ID
    user_ids = torch.randint(0, num_users, (batch_size,))
    # 生成用户性别
    user_genders = torch.randint(0, num_genders, (batch_size,))
    # 生成用户年龄
    user_ages = torch.randint(18, 60, (batch_size,)).float()

    # 生成商品 ID
    item_ids = torch.randint(0, num_items, (batch_size,))
    # 生成商品类别
    item_categories = torch.randint(0, num_categories, (batch_size,))
    # 生成商品价格
    item_prices = torch.rand(batch_size) * 1000

    # 生成用户的历史行为序列
    history_item_ids_list = []
    for _ in range(batch_size):
        seq_len = torch.randint(1, 6, (1,)).item()
        history_seq = torch.randint(0, num_items, (seq_len,))
        history_item_ids_list.append(history_seq)

    # 填充历史行为序列到相同长度
    max_len = max([seq.size(0) for seq in history_item_ids_list])
    padded_history_item_ids = torch.zeros((batch_size, max_len), dtype=torch.long)
    for i, seq in enumerate(history_item_ids_list):
        padded_history_item_ids[i, :seq.size(0)] = seq

    # 生成目标商品 ID
    target_item_ids = torch.randint(0, num_items, (batch_size,))
    # 生成点击标签
    labels = torch.randint(0, 2, (batch_size,)).float()

    return {
        'user_id': user_ids,
        'user_gender': user_genders,
        'user_age': user_ages.unsqueeze(1),
        'item_id': item_ids,
        'item_category': item_categories,
        'item_price': item_prices.unsqueeze(1),
        'history_item_ids': padded_history_item_ids,
        'target_item_id': target_item_ids,
        'label': labels
    }

# 调用 generate_sample_data 函数生成一个 batch 的数据
sample_data = generate_sample_data()
# 打印示例数据的键值
print("示例数据：", sample_data.keys())
# 打印历史行为序列的形状
print("历史行为序列形状:", sample_data['history_item_ids'].shape)


# 注意力池化层的实现 (标准 MLP 注意力网络)
class AttentionPoolingLayer(nn.Module):
    def __init__(self, embedding_dim):
        super(AttentionPoolingLayer, self).__init__()
        # 使用 MLP 注意力网络
        self.attention_fc = nn.Sequential(
            nn.Linear(embedding_dim * 2, embedding_dim), # 输入维度为 query 和 key 拼接后的维度
            nn.ReLU(),
            nn.Linear(embedding_dim, 1) # 输出维度为 1，表示注意力权重
        )

    def forward(self, queries, keys):
        """
        前向传播函数，计算注意力权重并进行加权池化
        :param queries: 目标商品的嵌入向量 (batch_size, embedding_dim)
        :param keys: 历史行为序列的嵌入向量 (batch_size, seq_len, embedding_dim)
        :return: 注意力池化后的用户兴趣表示 (batch_size, embedding_dim)
        """
        # 1. 计算注意力分数
        queries = queries.unsqueeze(1)  # (batch_size, 1, embedding_dim)
        # 扩展 queries 以便与 keys 进行拼接
        queries = queries.expand(-1, keys.size(1), -1) # (batch_size, seq_len, embedding_dim)
        # 将 query 和 key 拼接在一起
        attention_input = torch.cat([queries, keys], dim=-1) # (batch_size, seq_len, embedding_dim * 2)
        # 通过 MLP 注意力网络计算注意力分数
        attention_scores = self.attention_fc(attention_input.view(-1, attention_input.size(-1))).view(keys.size(0), keys.size(1)) # (batch_size, seq_len)

        # 2. 使用 Softmax 归一化注意力分数，得到注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1) # (batch_size, seq_len)

        # 3. 加权求和得到注意力池化后的结果
        attention_output = torch.sum(attention_weights.unsqueeze(-1) * keys, dim=1) # (batch_size, embedding_dim)

        return attention_output


class DIN(nn.Module):
    def __init__(self, num_users, num_items, num_categories, num_genders, embedding_dim, hidden_units):
        super(DIN, self).__init__()

        # 创建用户嵌入层
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        # 创建用户性别嵌入层
        self.user_gender_embedding = nn.Embedding(num_genders, embedding_dim)
        # 创建商品嵌入层
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        # 创建类别嵌入层
        self.category_embedding = nn.Embedding(num_categories, embedding_dim)

        # 创建数值型特征的线性层
        self.numerical_fc = nn.Linear(2, embedding_dim)

        # 创建注意力池化层
        self.attention_pooling = AttentionPoolingLayer(embedding_dim)

        # 创建全连接层
        fc_layers = []
        # 计算全连接层的输入维度 (6 个 embedding 特征)
        input_dim = embedding_dim * 6  # 用户，用户性别，商品，类别，数值型特征，注意力输出
        for units in hidden_units:
            fc_layers.append(nn.Linear(input_dim, units))
            fc_layers.append(nn.ReLU())
            input_dim = units
        fc_layers.append(nn.Linear(input_dim, 1))
        self.fc = nn.Sequential(*fc_layers)


    def forward(self, user_id, user_gender, user_age, item_id, item_category, item_price, history_item_ids, target_item_id):
        """
        DIN模型的前向传播函数

        输入:
            user_id: 用户ID, 形状: (batch_size,)
            user_gender: 用户性别, 形状: (batch_size,)
            user_age: 用户年龄, 形状: (batch_size, 1)
            item_id: 商品ID, 形状: (batch_size,)
            item_category: 商品类别, 形状: (batch_size,)
            item_price: 商品价格, 形状: (batch_size, 1)
            history_item_ids: 用户历史行为序列, 形状: (batch_size, seq_len)
            target_item_id: 目标商品ID, 形状: (batch_size,)

        输出:
            output: 点击率预测值, 形状: (batch_size,)
        """
        # 1. 获取各个特征的嵌入向量
        user_embed = self.user_embedding(user_id)
        user_gender_embed = self.user_gender_embedding(user_gender)
        item_embed = self.item_embedding(item_id)
        category_embed = self.category_embedding(item_category)
        target_item_embed = self.item_embedding(target_item_id)
        history_item_embed = self.item_embedding(history_item_ids)


        # 2. 处理数值型特征
        numerical_features = torch.cat([user_age, item_price], dim=-1)
        numerical_embed = F.relu(self.numerical_fc(numerical_features))

        # 3. 使用注意力池化层对历史行为进行加权
        attention_output = self.attention_pooling(queries=target_item_embed, keys=history_item_embed)

        # 4. 拼接所有特征
        concat_features = torch.cat([user_embed, user_gender_embed, item_embed, category_embed, numerical_embed, attention_output], dim=-1)

        # 5. 通过全连接层进行预测
        output = self.fc(concat_features)
        output = torch.sigmoid(output)

        return output.squeeze(1)


num_epochs = 10
batch_size = 64

# 初始化 DIN 模型
num_users = 1000
num_items = 2000
num_categories = 50
num_genders = 2
embedding_dim = 64
hidden_units = [128, 64]
model = DIN(num_users, num_items, num_categories, num_genders, embedding_dim, hidden_units)
print(model)

# 损失函数和优化器
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# LogLoss 计算函数 (PyTorch 版本)
def log_loss(y_true, y_pred):
    y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
    return -torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))

# 训练循环
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_idx in range(0, 1000, batch_size):
        train_batch_data = generate_sample_data(batch_size)
        labels = train_batch_data['label']

        optimizer.zero_grad()

        # 3. 前向传播
        predictions = model(train_batch_data['user_id'],
                          train_batch_data['user_gender'],
                          train_batch_data['user_age'],
                          train_batch_data['item_id'],
                          train_batch_data['item_category'],
                          train_batch_data['item_price'],
                          train_batch_data['history_item_ids'],
                          train_batch_data['target_item_id'])

        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / (1000 / batch_size)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")

print("训练完成!")

# 评估循环
def evaluate_model(model, batch_size=64):
    model.eval()
    all_labels = []
    all_predictions = []
    total_eval_loss = 0.0

    with torch.no_grad():
        for batch_idx in range(0, 500, batch_size):
            eval_batch_data = generate_sample_data(batch_size)
            labels = eval_batch_data['label']

            # 2. 前向传播
            predictions = model(eval_batch_data['user_id'],
                              eval_batch_data['user_gender'],
                              eval_batch_data['user_age'],
                              eval_batch_data['item_id'],
                              eval_batch_data['item_category'],
                              eval_batch_data['item_price'],
                              eval_batch_data['history_item_ids'],
                              eval_batch_data['target_item_id'])

            eval_loss = loss_fn(predictions, labels)
            total_eval_loss += eval_loss.item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    avg_eval_loss = total_eval_loss / (500 / batch_size)
    auc_score = roc_auc_score(all_labels, all_predictions)
    logloss_score = log_loss(torch.tensor(all_labels), torch.tensor(all_predictions)).item()

    print(f"Evaluation - Average Loss: {avg_eval_loss:.4f}, AUC: {auc_score:.4f}, LogLoss: {logloss_score:.4f}")
    return avg_eval_loss, auc_score, logloss_score


# 在训练完成后进行评估
print("开始评估...")
evaluate_model(model)
print("评估完成!")

示例数据： dict_keys(['user_id', 'user_gender', 'user_age', 'item_id', 'item_category', 'item_price', 'history_item_ids', 'target_item_id', 'label'])
历史行为序列形状: torch.Size([64, 5])
DIN(
  (user_embedding): Embedding(1000, 64)
  (user_gender_embedding): Embedding(2, 64)
  (item_embedding): Embedding(2000, 64)
  (category_embedding): Embedding(50, 64)
  (numerical_fc): Linear(in_features=2, out_features=64, bias=True)
  (attention_pooling): AttentionPoolingLayer(
    (attention_fc): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (fc): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)
Epoch [1/10], Average Loss: 1.2030
Epoch [2/10], Average Loss: 0.7499
Epoch [3/10], Average Loss: 0.7530
Epoch [4/10